In [42]:
import os
from tqdm import tqdm
import numpy as np
from xgboost import XGBRegressor
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

In [43]:
X_train = pd.read_pickle('./397/X_train_base.pkl').astype(float)
y_train = pd.read_pickle('./397/y_train_base.pkl').astype(float)
X_val = pd.read_pickle('./397/X_val_base.pkl').astype(float)
y_val = pd.read_pickle('./397/y_val_base.pkl').astype(float)
X_test = pd.read_pickle('./397/X_test_base.pkl').astype(float)
y_test = pd.read_pickle('./397/y_test_base.pkl').astype(float)

# XGBoost 

In [44]:
n_folds = 3
kfold = KFold(n_splits=n_folds, shuffle=True, random_state=0)
train_fold_predict = np.zeros((X_train.shape[0], 1))
test_predict = np.zeros((X_test.shape[0], n_folds))

In [45]:
xgboost_reg = XGBRegressor(n_estimators = 400,
                            learning_rate = 0.04,
                            colsample_bytree = 0.8,
                            subsample = 0.7 )

In [46]:
y_train

116913    0.006292
115458    0.005039
375236    0.011085
400420    0.004263
158032    0.002898
            ...   
213316    0.002781
220941    0.001824
378784    0.002106
402197    0.001141
247224    0.007090
Name: target, Length: 299050, dtype: float64

In [47]:
for cv_num, (train_index, val_index) in tqdm(enumerate(kfold.split(X_train))):
    X_train_ = X_train.iloc[train_index,:]
    y_train_ = y_train.iloc[train_index]
    X_val_ = X_train.iloc[val_index,:]

    xgboost_reg.fit(X_train_,y_train_)

    train_fold_predict[val_index,:] = xgboost_reg.predict(X_val_).reshape(-1,1)
    test_predict[:,cv_num] = xgboost_reg.predict(X_test)

xgb_test_predict_mean = np.mean(test_predict, axis=1).reshape(-1,1)
xgb_train_predict = train_fold_predict

3it [19:05, 381.83s/it]


In [48]:
xgb_test_predict_mean.shape

(85444, 1)

In [49]:
xgb_train_predict.shape

(299050, 1)

# CNN

In [50]:
X_train = pd.read_pickle('./397/X_train_base.pkl').astype(float)
y_train = pd.read_pickle('./397/y_train_base.pkl').astype(float)
X_val = pd.read_pickle('./397/X_val_base.pkl').astype(float)
y_val = pd.read_pickle('./397/y_val_base.pkl').astype(float)
X_test = pd.read_pickle('./397/X_test_base.pkl').astype(float)
y_test = pd.read_pickle('./397/y_test_base.pkl').astype(float)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.fit_transform(X_val)
X_test = scaler.fit_transform(X_test)

In [51]:
def np_rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true-y_pred)/y_true)))

In [52]:
def rmspe(y_true, y_pred):
    return K.sqrt(K.mean(K.square((y_true-y_pred)/y_true)))

In [53]:
def CNN(X_train, y_train, X_val, y_val, num_columns, num_labels, learning_rate, epochs):
    inp = tf.keras.layers.Input(shape=(num_columns,))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dense(256, activation='LeakyReLU')(x)
    x = tf.keras.layers.Reshape((16,16))(x)
    x = tf.keras.layers.Conv1D(filters=12, kernel_size=2, strides=1, activation='LeakyReLU')(x)
    x = tf.keras.layers.MaxPooling1D(pool_size=2)(x)
    x = tf.keras.layers.Flatten()(x)

    for i in range(3):
        x = tf.keras.layers.Dense(64//(2**i), activation='LeakyReLU')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.GaussianNoise(0.01)(x)
        x = tf.keras.layers.Dropout(0.20)(x)

    x = tf.keras.layers.Dense(num_labels)(x)

    model = tf.keras.models.Model(inputs=inp, outputs=x)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss=rmspe)

    rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_delta=1e-5, verbose=2, eps=1e-5)
    es = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=31, restore_best_weights=True, verbose=2)   
    
    history = model.fit(X_train, y_train, epochs=epochs,validation_data=(X_val, y_val), validation_batch_size=len(y_val),batch_size=batch_size,verbose=1, callbacks=[rlr,es])

    return model, history

In [54]:
y_train

116913    0.006292
115458    0.005039
375236    0.011085
400420    0.004263
158032    0.002898
            ...   
213316    0.002781
220941    0.001824
378784    0.002106
402197    0.001141
247224    0.007090
Name: target, Length: 299050, dtype: float64

In [55]:
tf.random.set_seed(777)
num_columns = X_train.shape[1]
num_labels = 1
learning_rate = 6e-3

tf.random.set_seed(777)
batch_size = 1024
dropout_rates = 0
epochs = 1000

n_folds = 3
kfold = KFold(n_splits=n_folds, shuffle=True, random_state=0)
train_fold_predict = np.zeros((X_train.shape[0], 1))
test_predict = np.zeros((X_test.shape[0], n_folds))

for cv_num, (train_index, val_index) in tqdm(enumerate(kfold.split(X_train))):
    X_train_ = X_train[train_index,:]
    y_train_ = y_train.iloc[train_index]
    X_val_ = X_train[val_index,:]

    model = CNN(X_train_, y_train_, X_val, y_val, num_columns, num_labels, learning_rate, epochs)

    train_fold_predict[val_index,:] = model[0].predict(X_val_).reshape(-1,1)
    test_predict[:,cv_num] = model[0].predict(np.array(X_test)).reshape(-1)

cnn_test_predict_mean = np.mean(test_predict, axis=1).reshape(-1,1)
cnn_train_predict = train_fold_predict

cnn_test_predict_mean.shape, cnn_train_predict.shape

0it [00:00, ?it/s]

Epoch 1/1000


2022-10-24 22:50:06.192798: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


195/195 [==============================] - ETA: 0s - loss: 36.8867

2022-10-24 22:50:10.923195: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


...

195/195 [==============================] - ETA: 0s - loss: 31.2943

2022-10-24 23:05:35.038726: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


195/195 [==============================] - 6s 24ms/step - loss: 31.2943 - val_loss: 0.7677
Epoch 2/1000
195/195 [==============================] - 4s 23ms/step - loss: 0.6356 - val_loss: 0.4617
Epoch 3/1000
195/195 [==============================] - 5s 24ms/step - loss: 0.6186 - val_loss: 0.4961
Epoch 4/1000
195/195 [==============================] - 4s 23ms/step - loss: 0.5297 - val_loss: 0.5772
Epoch 5/1000
195/195 [==============================] - 4s 22ms/step - loss: 0.5394 - val_loss: 0.3347
Epoch 6/1000
195/195 [==============================] - 4s 21ms/step - loss: 0.4571 - val_loss: 0.3658
Epoch 7/1000
195/195 [==============================] - 4s 22ms/step - loss: 0.5095 - val_loss: 0.4097
Epoch 8/1000
195/195 [==============================] - 4s 21ms/step - loss: 0.4438 - val_loss: 0.4305
Epoch 9/1000
195/195 [==============================] - 4s 21ms/step - loss: 0.4462 - val_loss: 0.6308
Epoch 10/1000
195/195 [==============================] - 4s 21ms/step - loss: 0.4499 

2022-10-24 23:10:39.918683: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
3it [20:45, 415.12s/it]


((85444, 1), (299050, 1))

# MLP

In [56]:
def np_rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true-y_pred)/y_true)))

In [57]:
def rmspe(y_true, y_pred):
    return K.sqrt(K.mean(K.square((y_true-y_pred)/y_true)))

In [58]:
def MLP(X_train, y_train, X_val, y_val):

    model = tf.keras.models.Sequential()
    model.add(
        tf.keras.layers.Dense(
            units=198,
            input_dim=X_train.shape[1],
            kernel_initializer='glorot_uniform',
            activation='ReLU'
        ))

    model.add(
        tf.keras.layers.Dense(
            units=49,
            input_dim=198,
            kernel_initializer='glorot_uniform',
            activation='ReLU'
        ))
        
    model.add(
        tf.keras.layers.Dense(
            units=1,
            input_dim=49,
            kernel_initializer='glorot_uniform',
    ))
    model.summary()
    model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
                loss=rmspe)

    rlr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_delta=1e-5, verbose=1)
    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=11, restore_best_weights=True, verbose=1)
    callback_list = [rlr, es]
    history = model.fit(X_train, y_train,
                        batch_size=500, epochs=1000, verbose=1,
                        validation_data=(X_val, y_val), callbacks=callback_list
    )

    return model

In [59]:
X_train[:100,0].shape

(100,)

In [60]:
tmp = X_train[:100,0].reshape(-1,1)

In [61]:
tmp.reshape(-1).shape

(100,)

In [64]:
tf.random.set_seed(777)

n_folds = 3
kfold = KFold(n_splits=n_folds, shuffle=True, random_state=0)
train_fold_predict = np.zeros((X_train.shape[0], 1))
test_predict = np.zeros((X_test.shape[0], n_folds))

for cv_num, (train_index, val_index) in tqdm(enumerate(kfold.split(X_train))):
    X_train_ = X_train[train_index,:]
    y_train_ = y_train.iloc[train_index]
    X_val_ = X_train[val_index,:]

    model = MLP(X_train_, y_train_, X_val, y_val)

    train_fold_predict[val_index,:] = model.predict(X_val_).reshape(-1,1)
    test_predict[:,cv_num] = model.predict(np.array(X_test)).reshape(-1)

mlp_test_predict_mean = np.mean(test_predict, axis=1).reshape(-1,1)
mlp_train_predict = train_fold_predict

0it [00:00, ?it/s]

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_45 (Dense)             (None, 198)               66528     
_________________________________________________________________
dense_46 (Dense)             (None, 49)                9751      
_________________________________________________________________
dense_47 (Dense)             (None, 1)                 50        
Total params: 76,329
Trainable params: 76,329
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000


2022-10-24 23:16:37.783978: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


399/399 [==============================] - ETA: 0s - loss: 23.1560

2022-10-24 23:16:42.529132: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


...

2022-10-24 23:22:43.683257: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


399/399 [==============================] - ETA: 0s - loss: 17.7086

2022-10-24 23:22:46.884076: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


399/399 [==============================] - 4s 8ms/step - loss: 17.7086 - val_loss: 0.4152
Epoch 2/1000
399/399 [==============================] - 3s 7ms/step - loss: 0.3249 - val_loss: 0.2928
Epoch 3/1000
399/399 [==============================] - 3s 7ms/step - loss: 0.2785 - val_loss: 0.2927
Epoch 4/1000
399/399 [==============================] - 3s 7ms/step - loss: 0.2691 - val_loss: 0.2761
Epoch 5/1000
399/399 [==============================] - 3s 8ms/step - loss: 0.2559 - val_loss: 0.3250
Epoch 6/1000
399/399 [==============================] - 3s 7ms/step - loss: 0.2517 - val_loss: 0.2656
Epoch 7/1000
399/399 [==============================] - 3s 7ms/step - loss: 0.2502 - val_loss: 0.2616
Epoch 8/1000
399/399 [==============================] - 3s 7ms/step - loss: 0.2417 - val_loss: 0.2567
Epoch 9/1000
399/399 [==============================] - 3s 7ms/step - loss: 0.2430 - val_loss: 0.2947
Epoch 10/1000
399/399 [==============================] - 3s 7ms/step - loss: 0.2498 - val_loss

2022-10-24 23:25:30.753202: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
3it [09:00, 180.28s/it]


In [65]:
new_X_train = np.concatenate((xgb_train_predict, cnn_train_predict, mlp_train_predict), axis=1)
new_X_test = np.concatenate((xgb_test_predict_mean, cnn_test_predict_mean, mlp_test_predict_mean), axis=1)

print(new_X_train.shape, new_X_test.shape)

(299050, 3) (85444, 3)


In [66]:
final_model = XGBRegressor()
final_model.fit(new_X_train, y_train)
y_pred_final = final_model.predict(new_X_test)

In [82]:
print(f'Final RMSPE: {rmspe(y_test, y_pred_final).numpy()}')

Final RMSPE: 0.2375852139618925
